# **Desenvolvimento de Projetos utilizando Redes Neurais**

## **Classificação**

O objetivo do projeto de classificação é construir um estimador que, tomando as características bioquímicas e genéticas de uma pessoa, possa estimar a probabilidade dessa pessoa ter diabetes nos 5 anos seguintes.

Link do dataset: https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names

In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# ignorar os warnings
warnings.filterwarnings('ignore')

In [2]:
# montar o drive
drive.mount('/content/drive/')

MessageError: ignored

In [ ]:
# carregar os dados
FILE_PATH = '/content/drive/MyDrive/Bootcamp_DataScience/RedesNeuraisTensorFlow/datasets/pima-indians-diabetes.data.csv'
data = pd.read_csv(FILE_PATH, header=None)

In [ ]:
data.head()

In [ ]:
# corrigir os nomes das colunas
data.columns = [
    'no_pregnants',
    'glucose_conc_2h',
    'blood_pressure_d',
    'triceps_thick',
    'insulin',
    'bmc',
    'diabetes_pedigree',
    'age',
    'class_diabetes'
]
data.head()

In [ ]:
# análise geral do dataset
data.info()

In [ ]:
# análise exploratória
data.describe()

In [ ]:
# analisando pessoas com dados inconsistentes
data.loc[data['blood_pressure_d'] == 0, :]

In [ ]:
# neste projeto - vamos eliminar as pessoas com dados inconsistentes
data = data.loc[data['blood_pressure_d'] != 0, :]
data.shape

In [ ]:
# nova análise descritiva
data.describe()

In [ ]:
# neste projeto - vamos eliminar as pessoas com dados inconsistentes
data = data.loc[data['triceps_thick'] != 0, :]
data.shape

In [ ]:
data.describe()

In [ ]:
# neste projeto - vamos eliminar as pessoas com dados inconsistentes
data = data.loc[data['bmc'] != 0, :]
data.shape

In [ ]:
data.describe()

In [ ]:
# análise de distribuição e correlação
sns.pairplot(data, hue='class_diabetes', palette='Set1')
plt.show()

- o conjunto de dados não é linearmente separável em nenhuma combinação bidimensional
- a variável que mais tem influência aparente no risco de diabetes (em 5 anos) é a  concentração de glicose no sangue após 2 h de teste - menor sobreposição de histogramas.

In [ ]:
# vamos analisar o balanceamento das classes
data['class_diabetes'].value_counts(normalize=True)

In [ ]:
# preparar os dados
# separar x e y
target = 'class_diabetes'
x = data.drop(columns=[target])
y = data[[target]]

In [ ]:
# separando conjuntos -treino, validação e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [ ]:
# verificando as proporções de classes
props = pd.DataFrame(data[target].value_counts(normalize=True).values,
                     index=data[target].value_counts(normalize=True).index,
                     columns=['original'])
props['treino'] = y_train[target].value_counts(normalize=True).values
props['val'] = y_val[target].value_counts(normalize=True).values
props['teste'] = y_test[target].value_counts(normalize=True).values
props

In [ ]:
# normalizar dados
scaler = MinMaxScaler(feature_range=(0, 1))

# treinar o scaler na base de treino (informação que temos)
scaler.fit(x_train)

# aplicar em todas as bases
x_train_norm = scaler.transform(x_train)
x_val_norm = scaler.transform(x_val)
x_test_norm = scaler.transform(x_test)

In [ ]:
# função para criar o modelo
def create_ann_model(x, n_neurons: list):
  """
  cria a rede neural com base em parâmetros informados
  """

  # definir o modelo
  model = Sequential()
  if len(n_neurons) == 1:
    model.add(Dense(n_neurons[0], input_shape=(8,), activation='relu'))
  else:
    for n in n_neurons:
      if n == n_neurons[0]:
        model.add(Dense(n, input_shape=(8,), activation='relu'))
      else:
        model.add(Dense(n, activation='relu'))

  # camada de saída - função de ativação sigmoidal
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# criando e testando o modelo
model1 = create_ann_model(x_train_norm, n_neurons=[15])
model1.summary()

In [ ]:
# compilar o modelo
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics='Recall')

# treinar o primeiro modelo
history = model1.fit(x_train_norm, y_train, epochs=200, batch_size=60, validation_data=(x_val_norm, y_val))

In [ ]:
def plot_history(history, n_epochs):
  fig = plt.figure(figsize=(12, 6))
  ax = fig.add_subplot(1, 2, 1)
  plt.plot(range(1, n_epochs+1), history.history['recall'], label='Treinamento')
  plt.plot(range(1, n_epochs+1), history.history['val_recall'], label='Validação')
  plt.legend(loc='best')
  plt.xlabel('Número de Épocas')
  plt.ylabel('Recall')
  plt.title('Curva de Aprendizado - Recall')

  ax = fig.add_subplot(1, 2, 2)
  plt.plot(range(1, n_epochs+1), history.history['loss'], label='Treinamento')
  plt.plot(range(1, n_epochs+1), history.history['val_loss'], label='Validação')
  plt.legend(loc='best')
  plt.xlabel('Número de Épocas')
  plt.ylabel('Função de Perda')
  plt.title('Curva de Aprendizado - Função de Perda')
  plt.show()

In [ ]:
plot_history(history, 200)

In [ ]:
# treinar o primeiro modelo com menos época e sem bateladas
history = model1.fit(x_train_norm, y_train, epochs=100, batch_size=x_train_norm.shape[0], validation_data=(x_val_norm, y_val))

In [ ]:
plot_history(history, 100)

In [ ]:
# propondo um modelo mais simples
model2 = create_ann_model(x_train_norm, n_neurons=[5])

# compilando e treinando
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics='Recall')

# treinar o primeiro modelo
history = model2.fit(x_train_norm, y_train, epochs=100, batch_size=60, validation_data=(x_val_norm, y_val))

In [ ]:
plot_history(history, 100)

- esse modelo mais simples não está conseguindo saber a diferença entre uma pessoa com diabetes e sem diabetes (ele prevê todas como não possuindo diabetes - apenas classes negativas)... portanto, não conseguimos calcular o recall. Vamos descartar esse modelo.

In [ ]:
# propondo um modelo mais simples, porém não tão simples
model3 = create_ann_model(x_train_norm, n_neurons=[10])

# compilando e treinando
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics='Recall')

# treinar o primeiro modelo
history = model3.fit(x_train_norm, y_train, epochs=100, batch_size=60, validation_data=(x_val_norm, y_val))

In [ ]:
plot_history(history, 100)

In [ ]:
# propondo um modelo mais simples, porém não tão simples
model3 = create_ann_model(x_train_norm, n_neurons=[10])

# compilando e treinando
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics='Recall')

# treinar o modelo
N_EPOCHS = 150
history = model3.fit(x_train_norm, y_train, epochs=N_EPOCHS, batch_size=60, validation_data=(x_val_norm, y_val))

plot_history(history, N_EPOCHS)

In [ ]:
# treinar o modelo
N_EPOCHS = 200
history = model3.fit(x_train_norm, y_train, epochs=N_EPOCHS, batch_size=60, validation_data=(x_val_norm, y_val))

plot_history(history, N_EPOCHS)

In [ ]:
# treinar o modelo
N_EPOCHS = 200
history = model3.fit(x_train_norm, y_train, epochs=N_EPOCHS, batch_size=30, validation_data=(x_val_norm, y_val))

plot_history(history, N_EPOCHS)

In [ ]:
# propondo um modelo mais profundo
model4 = create_ann_model(x_train_norm, n_neurons=[10, 10])

# compilando e treinando
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics='Recall')

# treinar o modelo
N_EPOCHS = 200
history = model4.fit(x_train_norm, y_train, epochs=N_EPOCHS, batch_size=60, validation_data=(x_val_norm, y_val))

plot_history(history, N_EPOCHS)

In [ ]:
# treinar o modelo
N_EPOCHS = 100
history = model4.fit(x_train_norm, y_train, epochs=N_EPOCHS, batch_size=60, validation_data=(x_val_norm, y_val))

plot_history(history, N_EPOCHS)

In [ ]:
# relembrando como é cada modelo treinado
print(model1.summary())
print(model2.summary())
print(model3.summary())
print(model4.summary())

- modelo 1 - 15 neurônios / 1 camada
- modelo 2 - 5 neurônios / 1 camada
- modelo 3 - 10 neurônios / 1 camada
- modelo 4 - 20 neurônios / 2 camadas (10 x 10)

In [ ]:
# função de análise de resultados
def analyze_results(x_test_norm, list_models, list_model_names, THRESHOLD=0.5):
  for i in range(len(list_models)):
    # realiza a previsão na base de teste
    yhat_test = list_models[i].predict(x_test_norm)

    # transforma em classes
    yhat_test_class = [1 if x > THRESHOLD else 0 for x in yhat_test]

    # imprime resultados
    print(f'DESEMPENHO DE CLASSIFICAÇÃO - MODELO {list_model_names[i]} - Prob.min = {THRESHOLD}')
    print(classification_report(y_test, yhat_test_class))
    print('*' * 50)
    print('\n')

In [ ]:
# definindo lista de modelos e de nomes
list_models = [model1, model2, model3, model4]
list_model_names = ['ann_15', 'ann_5', 'ann_10', 'ann_10_10']
analyze_results(x_test_norm, list_models, list_model_names)

In [ ]:
# testando um modelo mais conservador
analyze_results(x_test_norm, list_models, list_model_names, THRESHOLD=0.75)

In [ ]:
# testando um modelo mais liberal
analyze_results(x_test_norm, list_models, list_model_names, THRESHOLD=0.35)